In [1]:
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
import PIL

In [2]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

In [3]:
data=torchvision.datasets.ImageFolder("D:\\si\\Faces for Training\\Faces for Training",transform=transform)

In [4]:
device=torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [6]:
model=torchvision.models.vgg16(pretrained=True)

In [7]:
for i in model.parameters():
    i.requires_grad=False

In [8]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [9]:
for i in model.parameters():
    i.requires_grad=False

In [10]:
class no(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x

In [11]:
model.avgpool=nn.AdaptiveAvgPool2d((1,1))
model.classifier=nn.Sequential(
    
        nn.Linear(512,256),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5,inplace=False),
        nn.Linear(256,128)
    
    )

In [12]:
net=model.to(device)

In [13]:

class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        super().__init__()
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform
        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)

       
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] == img1_tuple[1]:
                    break
        else:

            while True:
                
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1] != img1_tuple[1]:
                    break

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])

        img0 = img0.convert("L")
        img1 = img1.convert("L")

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
        return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [14]:
sdata=SiameseNetworkDataset(data,transform)

In [15]:
data_loader=DataLoader(sdata,64,shuffle=True,num_workers=2)

In [ ]:
for i,l in data_loader:
    plt.imshow(i[0,0,:,:])
    print(l)
    break

In [16]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
       
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)

        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                    (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [18]:

criterion = ContrastiveLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.0005 )

In [ ]:
counter = []
loss_history = [] 
iteration_number= 0

for epoch in range(10):


    for i, (img0, img1, label) in enumerate(data_loader, 0):

       
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()

        
        optimizer.zero_grad()

        
        output1, output2 = net(img0, img1)

        
        loss_contrastive = criterion(output1, output2, label)

       
        loss_contrastive.backward()

        
        optimizer.step()

        
        if i % 50 == 0 :
            print(f"Epoch number {epoch}\n Current loss {loss_contrastive.item()}\n")
            iteration_number += 50

            counter.append(iteration_number)
            loss_history.append(loss_contrastive.item())

